In [1]:
#changes:
#     - 1_22_23 - used Service to call in webdriver without the deprecation warning.
#     - added check to look for null values in city,state, EEO columns

## Setup

In [2]:
#libraries needed to run code
from bs4 import BeautifulSoup

import pandas as pd, requests, time, random

from selenium import webdriver
#from selenium.webdriver.chrome.options import Options
#from selenium.webdriver.chrome.service import Service

#options = Options()
#options.add_argument("--headless")
#options.add_argument("--no-sandbox")

#causes deprecation warning
#options.headless = True


## Webscraper

In [4]:
# reads in the excel data sheet
# set workbook to excel file name 
workbook = 'AEPS 2023 Snapshot YM -copy.xlsx'
table = pd.read_excel(workbook,sheet_name = 'Snapshot')


city = table['City'].isnull().values.any()
state = table['State'].isnull().values.any()
EEO = table['EEO Job Number'].isnull().values.any()

if city or state or EEO:
  print("Check 'City', 'State', 'EEO Job Number' columns for blanks")
else:
  # set up for finding the proportion a city makes up in their state, per job group
  locations = table['City'] + ", " + table['State']
  job_number_city_state = table['EEO Job Number'].astype(str) + "," + table['City'] + "," + table['State']
  job_number_state = table['EEO Job Number'].astype(str)+ "," + table['State']
  job_number_city_state_count = job_number_city_state.map(job_number_city_state.value_counts())
  job_number_state_count = job_number_state.map(job_number_state.value_counts())
  df = pd.DataFrame([job_number_city_state,job_number_city_state_count,job_number_state,job_number_state_count]).T
  df['proportion'] = df[1]/df[3]
  df['Location'] = locations

  # gets unique list of locations that make up at least 5% of their state.
  unique_locations = df.loc[df['proportion'] >= 0.05]['Location'].unique()
  number_of_locations = len(unique_locations)
  print(f'There are {number_of_locations} locations to search for.')

There are 11 locations to search for.


Webscraping Portion

In [5]:
#prints estimated runtime
shortest = round(((number_of_locations * 1) + (number_of_locations * 3)) / 60,1)
longest = round(((number_of_locations * 3) + (number_of_locations * 5)) / 60,1)

print(f'Estimated runtime between {shortest} and {longest} minutes.')

#s = Service("/usr/bin/chromedriver")
wd = webdriver.Chrome('./chromedriver')

#dictionary that stores url for each city,state
sites = {}
for location in unique_locations:
  #gets html info from query search of location into census reporter
  url = "https://censusreporter.org/search/?q="
  wd.get(url+ location)
  time.sleep(random.randint(1, 3))
  #gets html page of search query for current location
  html = wd.execute_script("return document.documentElement.outerHTML")
  soup = BeautifulSoup(html)
  
  #checks to see if there is an h3 tag. If there isn't, that means that the city,state entered don't exist.
  #otherwise, get the first result from the search page.
  if soup.find('h3') == None:
    sites[location] = 'not found'
  else:
    location_site = soup.find('h3').find('a').get('href')
    sites[location] = location_site


#dictionary that stores the metro areas for each city,state
location_info = {}
test = {}
# gets the metro areas,county, or micro area if available. Return state if not.
for location in sites.keys():
  #for city,states that weren't found in the loop above
  if sites[location] == 'not found':
    location_info[location] = f'{location} was not found'
  else:
    wd.get(sites[location])
    time.sleep(random.randint(3, 5))
    html = wd.execute_script("return document.documentElement.outerHTML")
    soup = BeautifulSoup(html)
    #data stores all of the location info that is were the metro area is stored on census reporter
    data = soup.find('header',class_='column-full').find('p').find_all('a')
    test[location] = data
    #position is the dictionary used to keep track of where in list info certain information is
    position = {}
    info = []
    #for loop for getting the location text we want from the data variable
    for x,loc_info in enumerate(data):
      #each conditional statement checks to see at what position the metro/micro/county info is located and keeps track of it in dictionary position
      if 'Metro' in loc_info.text:
        position['Metro'] = x
      elif 'Micro' in loc_info.text:
        position['Micro'] = x
      elif 'County' in loc_info.text:
        position['County'] = x
      else:
        position[loc_info.text] = x
      info.append(loc_info.text)

#returns either metro, county, micro, or all available info depending on the following checks  
    if "Metro" in position:
      location_info[location] = info[position["Metro"]]
    elif "Micro" in position:
      location_info[location] = info[position['Micro']]
    elif "County" in position:
      location_info[location] = info[position['County']]
    else:
      location_info[location] = ",".join(info)

#dictionary containing all the locations and their metro areas
location_info

Estimated runtime between 0.7 and 1.5 minutes.


{'Ft. Worth, TX': 'Dallas-Fort Worth-Arlington, TX Metro Area',
 'Moffett Field, CA': 'Moffett Field, CA was not found',
 'Beltsville, MD': 'Washington-Arlington-Alexandria, DC-VA-MD-WV Metro Area',
 'Leander, TX': 'Austin-Round Rock-Georgetown, TX Metro Area',
 'Colorado Springs, CO': 'Colorado Springs, CO Metro Area',
 'Silverdale, WA': 'Bremerton-Silverdale-Port Orchard, WA Metro Area',
 'John Day Dam, OR': 'John Day Dam, OR was not found',
 'The Dalles, OR': 'The Dalles, OR Micro Area',
 'Triangle, VA': 'Washington-Arlington-Alexandria, DC-VA-MD-WV Metro Area',
 'Hooper, UT': 'Ogden-Clearfield, UT Metro Area',
 'Austin, TX': 'Austin-Round Rock-Georgetown, TX Metro Area'}

In [6]:
#function that maps metro areas to all locations (City,states) in the snapshot data. 
def metro_area(loc,loc_info):
  # if location in location_info dictionary, return its metro area. Else, return its State.
  return loc_info[loc] if loc in loc_info else loc.split(', ')[1]

#applying the function above to the snapshot data
table['Metro Area'] = df['Location'].apply(metro_area, args = [location_info])
table[['City','State','Metro Area']]

,City,State,Metro Area
0,Ft. Worth,TX,"Dallas-Fort Worth-Arlington, TX Metro Area"
1,Ft. Worth,TX,"Dallas-Fort Worth-Arlington, TX Metro Area"
2,Moffett Field,CA,"Moffett Field, CA was not found"
3,Ft. Worth,TX,"Dallas-Fort Worth-Arlington, TX Metro Area"
4,Beltsville,MD,"Washington-Arlington-Alexandria, DC-VA-MD-WV M..."
...,...,...,...
410,Moffett Field,CA,"Moffett Field, CA was not found"
411,Moffett Field,CA,"Moffett Field, CA was not found"
412,Moffett Field,CA,"Moffett Field, CA was not found"
413,Moffett Field,CA,"Moffett Field, CA was not found"


## Output

In [7]:
#outputs copy of snapshot data with addition of metro area column
table.to_excel("output.xlsx", index = False) 

Notes: 
 

1.   Sometimes the first result from the search query doesn't return the place but something else like the county info (i.e. Orange, CA). I need to find a way to access the place rather than rely on the first result for more accuracy
2.   Misspelled cities or cities that don't exist will return a 'not found'. For the time being, you have to look into those.
3. Need to fix time complexity on the second for loop of webscraping code
